In [1]:
from functions_mediaEval import handles_vars
from functions_mediaEval import InfoCluster
from functions_mediaEval import abrir_descriptores_mat
from functions_mediaEval import seleccionoDescriptoresNeg
from functions_mediaEval import GenerarClusteringVisual
from functions_mediaEval import seleccionImagenesPositivas
from functions_mediaEval import CalculoPesosModelo
from functions_mediaEval import AjusteMultiplesModelos
from functions_mediaEval import ClustersConImagenes
from functions_mediaEval import ActualizarProbabilidadesClusters
from functions_mediaEval import metodoOrdenarPorClusters
from functions_mediaEval import metodoMediaEval
from functions_mediaEval import saveMediaEvalResults

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import pandas as pd
from numpy import genfromtxt
import sklearn
import scipy.io
from sklearn.cluster import KMeans
import subprocess
import csv
import sys
from PIL import Image

### Comienza el programa 

#### Inicialización de variables y parámetros 

In [3]:
handles = handles_vars()

Selección de directorio de los datos

In [4]:
handles.Dir = r'D:\MediaEval2016\Dades'
handles.directorio = handles.Dir+"/"

Selección de algoritmo:
1: Visual
3: Multimedia
5: Relevance Feedback

In [5]:
handles.algoritmo = 3

Selección modelo para el ajuste: "logistic" , "vgg19"

In [6]:
handles.ajuste = "logistic"

Selección de tipo de Cluster para la selección de imágenes positivas:
1: Cluster Visual para seleccionar las imágenes positivas
2: Cluster Textual para seleccionar las imágenes positivas
3: 5 primeras imágenes de Flickr
4: Imágenes centroides a partir de las características visuales


In [7]:
tipoClusterSelPos = 1

Selección de tipo de Cluster para la selección del listado final de imágenes:
1: Cluster Visual
0: Cluster Textual

In [8]:
if handles.algoritmo==1:
    ordenCluster = 1
else:
    ordenCluster = 0

Selección de imágenes negativas:
1: Si hay listado
0: Si no hay listado

In [9]:
listado_negativas = 0

Selección del número de imágenes negativas a elegir

In [10]:
num_negativas = 10

Selección de algoritmo para la selección de las imágenes negativas:
1: Las últimas de Flicker
2: Las primeras imágenes de otro topic del mismo set (DevSet o TestSet)
3: Las negativas de distinto topic del listado de negativas
4: Las negativas del mismo topic del listado de negativas

In [11]:
tipo_negativas = 2

Elección del DevSet y del TestSet

In [21]:
topic_inici = 73
topic_final = 73

Generamos lista con los topics de cada set

In [22]:
if topic_inici>=1 and topic_inici<=70:
    DevSetTopics = scipy.io.loadmat('DevSetTopics.mat')
    DevSetTopics_list=[]
    for i in range(len(DevSetTopics["DevSetTopics"])):
        DevSetTopics_list.append([DevSetTopics["DevSetTopics"][i][0][0][0],DevSetTopics["DevSetTopics"][i][1][0]])
else:
    TestSetTopics = scipy.io.loadmat('TestSetTopics.mat')
    TestSetTopics_list=[]
    for i in range(len(TestSetTopics["TestSetTopics"])):
        TestSetTopics_list.append([TestSetTopics["TestSetTopics"][i][0][0][0],TestSetTopics["TestSetTopics"][i][1][0]])

Almacenamos los resultados en fichero indicado por MediaEval

In [23]:
team = "UNED_UV"
if handles.algoritmo == 1:
    run = "run1"
elif handles.algoritmo == 3:
    run = "run3"
elif handles.algoritmo == 5:
    run = "run5"
    
tipo = 'ini_' + str(topic_inici)
name = run + "_" + team + "_" + tipo + "_prueba72.txt"
filename = os.path.join(handles.Dir,"resultados",name)
fp = open(filename,"a")

Seleccionamos el tipo de distancia escogida para el error

In [24]:
handles.tipo_distancia = "seuclidean"

Seleccionamos el método para realizar el clustering. "kmeans18" o "kmeans_optimal"

In [25]:
handles.clustering = "kmeans18"

Seleccionamos el orden de iniciación:
1: Random
2: Score Textual

In [26]:
handles.init_order = 2

Definimos el directorio de R

In [27]:
handles.Rlocation = 'C:/Program Files/R/R-4.1.1/bin/Rscript'

Iniciamos algunas variables necesarias

In [28]:
handles.relevance0 = -1

#### Comienza el bucle

In [29]:
for topic in range(topic_inici,topic_final+1):
    if topic == 124:
        continue

    print(f"Procesando Topic {topic}\n")
    handles.topic = topic
    if topic>=1 and topic<=70:
        handles.nametopic = DevSetTopics_list[topic-1][1]
    elif topic>=71 and topic<=135:
        handles.nametopic = TestSetTopics_list[topic-1-70][1]
        
    #Cargamos los descriptores
    name_fmat = os.path.join(handles.Dir,"descriptores","topic"+str(topic)+".mat")
    names,datosImagenes,st,variances_pca = abrir_descriptores_mat(name_fmat)
    
    handles.st= st;
    handles.variances_pca = variances_pca
    
    #Ordenamos los datos
    
    fil,col = datosImagenes.shape
    ids_st = st[:,0]
    idsImagenes = datosImagenes[:,0]
    posis=[]
    for ids in ids_st:
        pos = np.argwhere(ids==idsImagenes)[0][1]
        posis.append(pos)
    
    handles.datosImagenes = datosImagenes[posis,:]
    handles.datos = handles.datosImagenes

    
    #Leemos el fichero de clustering
    
    dirClusters = os.path.join(handles.Dir,"clusterings/")
    theclusterings = os.listdir(dirClusters)
    enc = False
    for cluster in theclusterings:
        expregular = str(topic)+"-"
        if cluster.startswith(expregular):
            enc = True
            break
    if enc:
        datoscluster = pd.read_csv(os.path.join(dirClusters,cluster),sep="\t")
        handles.datosclusters = np.array(datoscluster)
    else:
        print("No se encuentra el fichero de clusterings")
    
    #En caso que hayamos seleccionado listado_neg=1 tendremos que abrir el fichero con las imagenes negativas
    
    #if listado_neg==1:
        #handles= leerListadoNeg(hObject=1,eventdata=1,handles);
    #end
    
    #Como hemos seleccionado generar las imagenes negativas, 
    #tipo_negativas = 2 Las primeras imágenes de otro topic del mismo set (DevSet o TestSet)
    handles,descNR = seleccionoDescriptoresNeg(handles,num_negativas,tipo_negativas)
    fil_neg,col_neg = descNR.shape
    #Per si no tenim tantes negatives com voliem
    if fil_neg<num_negativas:
        num_negativas = fil_neg
    
    #Ahora que tenemos las imágenes negativas, queremos seleccionar las positivas.
    if tipoClusterSelPos == 1 or tipoClusterSelPos ==2:
        handles = seleccionImagenesPositivas(handles,tipoClusterSelPos,listado_negativas)
    elif tipoClusterSelPos == 3:
        print("Todavia no esta implementado la seleccion de imagenes positivas tipo 3")
    elif tipoClusterSelPos == 4:
        print("Todavia no esta implementado la seleccion de imagenes positivas tipo 4")
    
    filas,colum = handles.datos.shape
    filas_queries,colum_queries = handles.queries.shape
    
    #Actualizamos las variables de ImaRel
    handles.imaRel = handles.relevance0*np.ones((filas+filas_queries,1))
    handles.imaRel[filas:] = 0
    tam = handles.imaRel.shape[0]
    handles.imaRelHistorico = -1*np.ones((tam,1))
    handles.imaProb = np.ones((filas+filas_queries,1))
    handles.imaProb1 = np.ones((filas+filas_queries,1))
    handles.imaProb2 = np.ones((filas+filas_queries,1))
    handles.imaProbHistoria = []
    
    #Cambio los id de las queries para diferenciarlas de su copia en handles.datos
    maxi = handles.datos[:,0].max()
    for i in range(filas_queries):
        handles.queries[i,0] = maxi+1+i
        
    #Añadimos los descriptores negativos al principio de handles.datos
    handles.datosyneg = np.vstack((descNR,handles.datos))
    handles.negativas = descNR
    
    #Aqui se podria hacer pca pero no la hacemos
    
    #Calculamos la distancia entre todas las imagenes, para ello añadimos las queries con sus descriptores al final de los datos
    handles.datosyqueries = np.vstack((handles.datos,handles.queries))
    losdescrip = handles.datosyqueries[:,2:]
    
    #Creo copias de datos y datosyqueries por si luego se modifican no afectar a las originales
    handles.datosorden = handles.datos
    handles.datosyqueriesorden = np.vstack((handles.datos,handles.queries))
    
    #Iniciamos las varibles segun 1: Aleatorio 2: Por el score textual
    #if handles.init_order == 1:
        #print("No se ha desarrollado el init_order = 1")
    #elif handles.init_order = 2:
        #Al estar ordenados no tiene mucho sentido
    
    #COMIENZA EL ALGORITMO
    if handles.algoritmo == 1: #Solo visual + cluster visual + regresion lineal
        print("Este algoritmo no está implementado")
        #metodoMediaEval(handles,ordenCluster)
    elif handles.algoritmo == 3: #Multimedia OrdenCluster (textual o visual)
        print("Comienza el algoritmo")
        handles = metodoMediaEval(handles,ordenCluster)
        print("Algoritmo terminado")
    elif handles.algoritmo == 5:
        print("Este algoritmo no está implementado\n")
        
    #Guardamos los resultados
    saveMediaEvalResults(handles,fp)

fp.close()

Procesando Topic 73

Comienza el algoritmo
v_datos: (315, 50)


lista_imag_rel: (315, 1)


X_pos: (5, 50)

X_neg: (10, 50)

y_pos: (5,)

y_neg: (10,)

X_train: (15, 50)

y_train: (15,)

Probabilidades test: (300, 2)

[[1.59694996e-03 9.98403050e-01]
 [2.70598929e-03 9.97294011e-01]
 [9.82248063e-04 9.99017752e-01]
 [4.31959245e-03 9.95680408e-01]
 [1.95359629e-03 9.98046404e-01]
 [2.76596101e-03 9.97234039e-01]
 [2.88492340e-03 9.97115077e-01]
 [1.52662523e-03 9.98473375e-01]
 [8.68974189e-03 9.91310258e-01]
 [3.20327263e-04 9.99679673e-01]
 [3.24896863e-02 9.67510314e-01]
 [9.69595557e-04 9.99030404e-01]
 [3.23141561e-03 9.96768584e-01]
 [8.90148197e-02 9.10985180e-01]
 [6.77380918e-04 9.99322619e-01]
 [3.48944202e-03 9.96510558e-01]
 [3.20247723e-03 9.96797523e-01]
 [1.40757239e-03 9.98592428e-01]
 [2.54492859e-01 7.45507141e-01]
 [7.24718307e-02 9.27528169e-01]
 [9.21036966e-03 9.90789630e-01]
 [3.06774866e-01 6.93225134e-01]
 [2.46687618e-01 7.53312382e-01]
 [4.87975375e-01 5.12024

UnboundLocalError: local variable 'v_componentesModelo' referenced before assignment

In [22]:
a = np.ones(10)
a[:5]=0

In [23]:
a[a==1]

array([1., 1., 1., 1., 1.])

In [24]:
a[np.where(a==0)]

array([0., 0., 0., 0., 0.])

In [25]:
v_datos = handles.datosyqueriesynegativasorden[:,2:]

In [26]:
lista_imag_rel = np.ones((315,1))*-1

In [29]:
np.where(lista_imag_rel==-1)[0].shape

(315,)